In [ ]:
def get_sample(sample_max_size = int(2e32), random_size = True, element_range=(0, 101), value_type=int):
    from numpy.random import randint

    sample_size = randint(0, sample_max_size) if random_size else sample_max_size
    return randint(low=element_range[0], high=element_range[1], size=sample_size, dtype=value_type)

In [ ]:
def clf_function(a, b):
    return a > b

In [ ]:
def bubble_sort(elements, clf_function):
    
    length = len(elements)
    for i in range(length):
        swapped = False
        for j in range(length - i -1):
            if clf_function(elements[j], elements[j+1]):
                elements[j], elements[j+1] = elements[j+1], elements[j]
                swapped = True
        if not swapped:
            break
    return elements

In [ ]:
def performance_test(sample, clf_function):
    from time import process_time
    from collections import namedtuple

    built_in_time_0 = process_time()
    built_in_sorted_list = sorted(sample)
    built_in_delta = process_time() - built_in_time_0

    comparison_time_0 = process_time()
    comparison_sorted_list = bubble_sort(sample, clf_function)
    comparison_delta = process_time() - comparison_time_0
    
    Result_test = namedtuple('Test', 'elements, delta_time')
    built_in = Result_test(built_in_sorted_list, built_in_delta)
    cmp = Result_test(comparison_sorted_list, comparison_delta)
    
    report = f'Built in: {built_in_delta:.4f}\n' + \
             f'Comparison: {comparison_delta:.4f}\n' + \
             f'Relative: {comparison_delta / built_in_delta:.4f}'
             
    return (built_in, cmp, report)

In [ ]:
def effectiveness_test(pattern, sample, quiet_mode=False):
    if any(pattern != sample):
        for p, s in zip(pattern, sample):
            if p != s and not quiet_mode:
                print("{}/{}".format(p, s))
        return False
    else:
        if not quiet_mode:
            print("Success")
        return True

In [ ]:
sample_size = 1024
print("Running a sample with {} element{}.".
      format(sample_size, 's' if sample_size > 1 else ''))

sample = get_sample(sample_size, random_size=False)
pattern, sample, result = performance_test(sample, clf_function)

if effectiveness_test(pattern.elements, sample.elements):
    print(result)

## Running 2048 times a sample with 1024 elements

In [ ]:
sample_size = 1024
test_size = 2048 
print("Running {} times a sample with {} element{}.".
      format(test_size, sample_size, 's' if sample_size > 1 else ''))

errors = 0
built_in_time = []
comparison_time = []

for test in range(test_size):
    sample = get_sample(sample_size, random_size=False)
    pattern, sample, result = performance_test(sample, clf_function)
    if effectiveness_test(pattern.elements, sample.elements, quiet_mode=True):
        built_in_time.append(pattern.delta_time)
        comparison_time.append(sample.delta_time)
    else:
        errors += 1
        
print("In the end of the day {} error{} occurred".format(errors, "s" if errors > 1 else ""))

In [ ]:
from scipy import stats

built_in_summary = stats.describe(built_in_time)
comparison_summary = stats.describe(comparison_time)
print('Built in average time: {:13.4f}\n'
      'Comparison average time: {:11.4f}\n'
      'Relative mean efficience: {:>11.4f}'.
      format(
            built_in_summary.mean,
            comparison_summary.mean,
            comparison_summary.mean / built_in_summary.mean
       )
)

print('-' * 80)

print('Built in variance time: {:20.12f}\n'
      'Comparison variance time: {:18.12f}\n'
      'Relative variance: {:>28.12f}'.
      format(
            built_in_summary.variance,
            comparison_summary.variance,
            comparison_summary.variance / built_in_summary.variance
       )
)

In [ ]:
import matplotlib.pyplot as plt
%matplotlib inline


# built in sort method
plt.subplot(221)
plt.xlabel("{} runs of sample Size: {}".format(test_size, sample_size))
plt.ylabel("$Elapsed \quad  Time$")
plt.title("Built in Sort")
plt.plot(
    built_in_time, "b{}-".format("o" if test_size < 25 else "")
)
maximal = max(built_in_time)
minimal = min(built_in_time)
plt.plot(
    [maximal] * test_size, "{}-".format("o" if test_size < 25 else ""),
    label="Max: {:.6f}".format(maximal)
)
plt.plot(
    [minimal] * test_size, "{}-".format("o" if test_size < 25 else ""),
    label="Min: {:.6f}".format(minimal)
)
plt.legend(bbox_to_anchor=(1.05, 1), loc=2, borderaxespad=0.)
plt.axis([-.1, test_size - .9, 0, maximal * 1.5])
plt.grid(True)


# Bubble sort
plt.subplot(222)
plt.xlabel("{} runs of sample Size: {}".format(test_size, sample_size))
plt.ylabel("$Elapsed \quad  Time$")
plt.title("Bubble sort")
plt.plot(
    comparison_time, "r{}-".format("o" if test_size < 25 else ""),
)
maximal = max(comparison_time)
minimal = min(comparison_time)
plt.plot(
    [maximal] * test_size, "{}-".format("o" if test_size < 25 else ""),
    label="Max: {:.6f}".format(maximal)
)
plt.plot(
    [minimal] * test_size, "{}-".format("o" if test_size < 25 else ""),
    label="Min: {:.6f}".format(minimal)
)
plt.legend(bbox_to_anchor=(1.05, 1), loc=2, borderaxespad=0.)
plt.axis([-.1, test_size - .9, 0, maximal * 1.5])
plt.grid(True)


# Both Bubble and Built in sort
plt.subplot(223)
plt.xlabel("{} runs of sample Size: {}".format(test_size, sample_size))
plt.ylabel("$Elapsed \quad  Time$")
plt.title("Relative performance")
plt.plot(
    built_in_time, "b{}-".format("o" if test_size < 25 else ""), label="Built in Sort"
)
plt.plot(
    comparison_time, "r{}-".format("o" if test_size < 25 else ""), label="Bubble Sort"
)
plt.legend(loc="upper right")
maximal = max(comparison_time + built_in_time)
plt.axis([-.1, test_size - .9, 0, maximal * 1.5])
plt.grid(True)


# Ratio between Bubble Sort / Built in Sort
plt.subplot(224)
plt.xlabel("{} runs of sample Size: {}".format(test_size, sample_size))
plt.ylabel("$Relative \quad Elapsed \quad  Time$")
plt.title("Bubble sort / Built in Sort")
ratios = [bubble / cmp for bubble, cmp in zip(comparison_time, built_in_time)]
plt.plot(ratios, "g{}-".format("o" if test_size < 25 else ""))

maximal = max(ratios)
minimal = min(ratios)
plt.plot(
    [maximal] * test_size, "{}-".format("o" if test_size < 25 else ""),
    label="Max: {:.6f}".format(maximal)
)
plt.plot(
    [minimal] * test_size, "{}-".format("o" if test_size < 25 else ""),
    label="Min: {:.6f}".format(minimal)
)
plt.plot(
    [1] * test_size, "{}-".format("o" if test_size < 25 else ""), label="1"
)
plt.legend(bbox_to_anchor=(1.05, 1), loc=2, borderaxespad=0.)

plt.axis([-.1, test_size - .9, 0, maximal * 1.5])
plt.grid(True)


plt.subplots_adjust(top=1.5, bottom=0, left=0.10, right=1.95, hspace=.5, wspace=.75)
plt.show()

## Running 2048 times a sample with 2048 elements

In [ ]:
sample_size = 2048
test_size = 2048 
print("Running {} times a sample with {} element{}.".
      format(test_size, sample_size, 's' if sample_size > 1 else ''))

errors = 0
built_in_time = []
comparison_time = []

for test in range(test_size):
    sample = get_sample(sample_size, random_size=False)
    pattern, sample, result = performance_test(sample, clf_function)
    if effectiveness_test(pattern.elements, sample.elements, quiet_mode=True):
        built_in_time.append(pattern.delta_time)
        comparison_time.append(sample.delta_time)
    else:
        errors += 1
        
print("In the end of the day {} error{} occurred".format(errors, "s" if errors > 1 else ""))

In [ ]:
built_in_summary = stats.describe(built_in_time)
comparison_summary = stats.describe(comparison_time)
print('Built in average time: {:13.4f}\n'
      'Comparison average time: {:11.4f}\n'
      'Relative mean efficience: {:>11.4f}'.
      format(
            built_in_summary.mean,
            comparison_summary.mean,
            comparison_summary.mean / built_in_summary.mean
       )
)

print('-' * 80)

print('Built in variance time: {:20.12f}\n'
      'Comparison variance time: {:18.12f}\n'
      'Relative variance: {:>28.12f}'.
      format(
            built_in_summary.variance,
            comparison_summary.variance,
            comparison_summary.variance / built_in_summary.variance
       )
)

In [ ]:
# built in sort method
plt.subplot(221)
plt.xlabel("{} runs of sample Size: {}".format(test_size, sample_size))
plt.ylabel("$Elapsed \quad  Time$")
plt.title("Built in Sort")
plt.plot(
    built_in_time, "b{}-".format("o" if test_size < 25 else "")
)
maximal = max(built_in_time)
minimal = min(built_in_time)
plt.plot(
    [maximal] * test_size, "{}-".format("o" if test_size < 25 else ""),
    label="Max: {:.6f}".format(maximal)
)
plt.plot(
    [minimal] * test_size, "{}-".format("o" if test_size < 25 else ""),
    label="Min: {:.6f}".format(minimal)
)
plt.legend(bbox_to_anchor=(1.05, 1), loc=2, borderaxespad=0.)
plt.axis([-.1, test_size - .9, 0, maximal * 1.5])
plt.grid(True)


# Bubble sort
plt.subplot(222)
plt.xlabel("{} runs of sample Size: {}".format(test_size, sample_size))
plt.ylabel("$Elapsed \quad  Time$")
plt.title("Bubble sort")
plt.plot(
    comparison_time, "r{}-".format("o" if test_size < 25 else ""),
)
maximal = max(comparison_time)
minimal = min(comparison_time)
plt.plot(
    [maximal] * test_size, "{}-".format("o" if test_size < 25 else ""),
    label="Max: {:.6f}".format(maximal)
)
plt.plot(
    [minimal] * test_size, "{}-".format("o" if test_size < 25 else ""),
    label="Min: {:.6f}".format(minimal)
)
plt.legend(bbox_to_anchor=(1.05, 1), loc=2, borderaxespad=0.)
plt.axis([-.1, test_size - .9, 0, maximal * 1.5])
plt.grid(True)


# Both Bubble and Built in sort
plt.subplot(223)
plt.xlabel("{} runs of sample Size: {}".format(test_size, sample_size))
plt.ylabel("$Elapsed \quad  Time$")
plt.title("Relative performance")
plt.plot(
    built_in_time, "b{}-".format("o" if test_size < 25 else ""), label="Built in Sort"
)
plt.plot(
    comparison_time, "r{}-".format("o" if test_size < 25 else ""), label="Bubble Sort"
)
plt.legend(loc="upper right")
maximal = max(comparison_time + built_in_time)
plt.axis([-.1, test_size - .9, 0, maximal * 1.5])
plt.grid(True)


# Ratio between Bubble Sort / Built in Sort
plt.subplot(224)
plt.xlabel("{} runs of sample Size: {}".format(test_size, sample_size))
plt.ylabel("$Relative \quad Elapsed \quad  Time$")
plt.title("Bubble sort / Built in Sort")
ratios = [bubble / cmp for bubble, cmp in zip(comparison_time, built_in_time)]
plt.plot(ratios, "g{}-".format("o" if test_size < 25 else ""))

maximal = max(ratios)
minimal = min(ratios)
plt.plot(
    [maximal] * test_size, "{}-".format("o" if test_size < 25 else ""),
    label="Max: {:.6f}".format(maximal)
)
plt.plot(
    [minimal] * test_size, "{}-".format("o" if test_size < 25 else ""),
    label="Min: {:.6f}".format(minimal)
)
plt.plot(
    [1] * test_size, "{}-".format("o" if test_size < 25 else ""), label="1"
)
plt.legend(bbox_to_anchor=(1.05, 1), loc=2, borderaxespad=0.)

plt.axis([-.1, test_size - .9, 0, maximal * 1.5])
plt.grid(True)


plt.subplots_adjust(top=1.5, bottom=0, left=0.10, right=1.95, hspace=.5, wspace=.75)
plt.show()